#### In deze notebook zullen we de voor IJsbaan proberen te bepalen wat het goedkoopste optie was mbt contracten (variabel versus markt)

Hiervoor zullen we de volgende vragen gaan betantwoorden:

Wat is goedkoper voor de klant?

A.	<em>Afrekening via dynamische propositie, dus werkelijk verbruik per uur i.c.m. met de marktprijs</em>

B.	<em>Afrekening via gewogen gemiddelde (profiel) propositie, dus alleen het totale verbruik nemen en dat profileren over de periode (december en januari apart)</em>

we halen nu de data op uit sql met de volgende query: 
<code>
SELECT 
    FORMAT(TR.StartDate, 'dd-MM-yyyy HH:mm') as 'Start datum', 
    FORMAT(TR.EndDate, 'dd-MM-yyyy HH:mm') as 'Eind datum', 
    TR.UsageLDN as 'Verbruik in kW/h' 
FROM 
    CRM.TelemetryReading TR
INNER JOIN 
    CRM.Connection C ON C.EAN = TR.EAN
WHERE 
    TR.EAN = '871685920002580359'
    AND TR.StartDate >= '2023-12-01' 
    AND TR.StartDate <= '2024-01-31' 
	--AND TR.UsageLDN != 0
ORDER BY
    TR.StartDate ASC; 
</code>   


In [94]:
import pandas as pd

In [95]:
df = pd.read_csv(r"telemetry.csv")
df.head()

,Start datum,Eind datum,Verbruik in kW/h
0,01-12-2023 01:00,01-12-2023 01:15,0.0
1,01-12-2023 01:15,01-12-2023 01:30,0.0
2,01-12-2023 01:30,01-12-2023 01:45,0.0
3,01-12-2023 01:45,01-12-2023 02:00,0.0
4,01-12-2023 02:00,01-12-2023 02:15,0.0


In [96]:
# we zullen eerst met pd de data naar dt omzetten.
df['Start datum'] = pd.to_datetime(df['Start datum'], format='%d-%m-%Y %H:%M')
df['Eind datum'] = pd.to_datetime(df['Eind datum'], format='%d-%m-%Y %H:%M')

# een nieuwe kolom maken voor de aangepaste tijd
df['Adjusted Hour'] = df['Start datum'].dt.floor('h')

# vervolgens berkenen we de som per uur
sum_per_hour = df.groupby('Adjusted Hour')['Verbruik in kW/h'].sum().reset_index()

# nu zetten we de aangepaste tijd weer om naar hoe we het eerst hadden
sum_per_hour['Start datum'] = sum_per_hour['Adjusted Hour']
sum_per_hour['Eind datum'] = sum_per_hour['Adjusted Hour'] + pd.Timedelta(hours=1)

# vebruik in kwh naar totaal verbruik per kwh/h
sum_per_hour.rename(columns={'Verbruik in kW/h': 'Totaal Verbruik in kW/h'}, inplace=True)

# en nu hebben we een nieuwe df waarbij we per uur het vebruik hebben ipv per kwartier
ijsbaan_uur = sum_per_hour[['Start datum', 'Eind datum', 'Totaal Verbruik in kW/h']]

#we pakken even een random rij om te bekijken
print(ijsbaan_uur.iloc[150])

# en nu printen we de hele mikmak in uit in onze nieuwe df genaamd ijsbaan_uur
print(ijsbaan_uur)



Start datum                2023-12-07 07:00:00
Eind datum                 2023-12-07 08:00:00
Totaal Verbruik in kW/h                 55.488
Name: 150, dtype: object
             Start datum          Eind datum  Totaal Verbruik in kW/h
0    2023-12-01 01:00:00 2023-12-01 02:00:00                      0.0
1    2023-12-01 02:00:00 2023-12-01 03:00:00                      0.0
2    2023-12-01 03:00:00 2023-12-01 04:00:00                      0.0
3    2023-12-01 04:00:00 2023-12-01 05:00:00                      0.0
4    2023-12-01 05:00:00 2023-12-01 06:00:00                      0.0
...                  ...                 ...                      ...
1460 2024-01-30 21:00:00 2024-01-30 22:00:00                      0.0
1461 2024-01-30 22:00:00 2024-01-30 23:00:00                      0.0
1462 2024-01-30 23:00:00 2024-01-31 00:00:00                      0.0
1463 2024-01-31 00:00:00 2024-01-31 01:00:00                      0.0
1464 2024-01-31 01:00:00 2024-01-31 02:00:00                    

#### marktprijzen


We gaan nu de marktprijzen zoeken voor dezelfde momenten zodat we de dataset kunnnen gaan joinen om een marktprijs berekening mogelijk te maken. We zullen ook alvast de data (start & end date) naar dt verzetten zodat het mogelijk is om te joinen.

hiervoor gebruiken we het resultaat van de volgende query:
<code>
SELECT 
	FORMAT(MP.StartDate, 'dd-MM-yyyy HH:mm') as 'Start datum', 
    FORMAT(MP.EndDate, 'dd-MM-yyyy HH:mm') as 'Eind datum',
	Price / 1000 as PricePerKwH
FROM 
	CRM.MarketPrice as MP
WHERE 
	CONVERT(date, PriceDate AT TIME ZONE 'Central European Standard Time') >= '2023-12-01'
	AND CONVERT(date, PriceDate AT TIME ZONE 'Central European Standard Time') < '2024-01-31'
	AND MarketPriceType IN ('ApxDayAheadHour')
ORDER BY
	MP.StartDate ASC
</code>



In [97]:
mp = pd.read_csv(r"marktprijs_uur.csv")

mp['Start datum'] = pd.to_datetime(df['Start datum'], format='%d-%m-%Y %H:%M')
mp['Eind datum'] = pd.to_datetime(df['Eind datum'], format='%d-%m-%Y %H:%M')

print(mp)

             Start datum          Eind datum  PricePerKwH
0    2023-12-01 01:00:00 2023-12-01 01:15:00      0.10691
1    2023-12-01 01:15:00 2023-12-01 01:30:00      0.10263
2    2023-12-01 01:30:00 2023-12-01 01:45:00      0.09910
3    2023-12-01 01:45:00 2023-12-01 02:00:00      0.09589
4    2023-12-01 02:00:00 2023-12-01 02:15:00      0.09607
...                  ...                 ...          ...
1411 2023-12-15 17:45:00 2023-12-15 18:00:00      0.07016
1412 2023-12-15 18:00:00 2023-12-15 18:15:00      0.06861
1413 2023-12-15 18:15:00 2023-12-15 18:30:00      0.05757
1414 2023-12-15 18:30:00 2023-12-15 18:45:00      0.05966
1415 2023-12-15 18:45:00 2023-12-15 19:00:00      0.05600

[1416 rows x 3 columns]


we zien dat de marktprijs per uur beschikbaar is en dat de telemetry data per kwartier is geveven daarom zullen we een sum van het vebruik doen zodat we per uur een verbruik hebben.

In [98]:
#dan gaan we nu door naar de grote join show:

final_df = ijsbaan_uur.join(mp, how='inner', lsuffix='ijsbaan_uur', rsuffix='mp')
final_df['Total Cost'] = final_df['Totaal Verbruik in kW/h'] * final_df['PricePerKwH']
final_df


,Start datumijsbaan_uur,Eind datumijsbaan_uur,Totaal Verbruik in kW/h,Start datummp,Eind datummp,PricePerKwH,Total Cost
0,2023-12-01 01:00:00,2023-12-01 02:00:00,0.0,2023-12-01 01:00:00,2023-12-01 01:15:00,0.10691,0.0
1,2023-12-01 02:00:00,2023-12-01 03:00:00,0.0,2023-12-01 01:15:00,2023-12-01 01:30:00,0.10263,0.0
2,2023-12-01 03:00:00,2023-12-01 04:00:00,0.0,2023-12-01 01:30:00,2023-12-01 01:45:00,0.09910,0.0
3,2023-12-01 04:00:00,2023-12-01 05:00:00,0.0,2023-12-01 01:45:00,2023-12-01 02:00:00,0.09589,0.0
4,2023-12-01 05:00:00,2023-12-01 06:00:00,0.0,2023-12-01 02:00:00,2023-12-01 02:15:00,0.09607,0.0
...,...,...,...,...,...,...,...
1411,2024-01-28 20:00:00,2024-01-28 21:00:00,0.0,2023-12-15 17:45:00,2023-12-15 18:00:00,0.07016,0.0
1412,2024-01-28 21:00:00,2024-01-28 22:00:00,0.0,2023-12-15 18:00:00,2023-12-15 18:15:00,0.06861,0.0
1413,2024-01-28 22:00:00,2024-01-28 23:00:00,0.0,2023-12-15 18:15:00,2023-12-15 18:30:00,0.05757,0.0
1414,2024-01-28 23:00:00,2024-01-29 00:00:00,0.0,2023-12-15 18:30:00,2023-12-15 18:45:00,0.05966,0.0


In [99]:
# dan nu de laatste stap; optellen van de totale kosten.
Totaal_kosten_marktprijs = final_df['Total Cost'].sum()
print(f"De totale kosten voor de periode in december en januari dat er werd vebruikt zijn: €{Totaal_kosten_marktprijs}")

De totale kosten voor de periode in december en januari dat er werd vebruikt zijn: €4415.49816348


#### De tweede vraag

De tweede vraag die we ons stellen is wat de prijs zou zijn geweest als we een profiel hadden gevolgd in dezelfde periode. daarvoor zullen we kijken naar dezelfde periode. 

hier nogmaals de vraag: 


B.	<em>Afrekening via gewogen gemiddelde (profiel) propositie, dus alleen het totale verbruik nemen en dat profileren over de periode (december en januari apart)</em>

hiervoor zullen we eerst de dataset die we hadden (telemetery) splitten in een december en in een januari dataset.

In [114]:
df_dec = df[df['Start datum'].dt.month == 12]
df_dec = df_dec.drop(['Adjusted Hour'], axis=1)
df_dec

,Start datum,Eind datum,Verbruik in kW/h
0,2023-12-01 01:00:00,2023-12-01 01:15:00,0.000
1,2023-12-01 01:15:00,2023-12-01 01:30:00,0.000
2,2023-12-01 01:30:00,2023-12-01 01:45:00,0.000
3,2023-12-01 01:45:00,2023-12-01 02:00:00,0.000
4,2023-12-01 02:00:00,2023-12-01 02:15:00,0.000
...,...,...,...
2967,2023-12-31 22:45:00,2023-12-31 23:00:00,20.812
2968,2023-12-31 23:00:00,2023-12-31 23:15:00,18.438
2969,2023-12-31 23:15:00,2023-12-31 23:30:00,17.862
2970,2023-12-31 23:30:00,2023-12-31 23:45:00,21.462


totaal vebruik voor in januari berekenen

In [115]:
df_dec_totaal = df_dec['Verbruik in kW/h'].sum()
print(df_dec_totaal)

46650.954


In [112]:
df_jan = df[df['Start datum'].dt.month == 1]
df_jan = df_jan.drop(['Adjusted Hour'], axis=1)
df_jan

,Start datum,Eind datum,Verbruik in kW/h
2972,2024-01-01 00:00:00,2024-01-01 00:15:00,22.0
2973,2024-01-01 00:15:00,2024-01-01 00:30:00,22.0
2974,2024-01-01 00:30:00,2024-01-01 00:45:00,23.0
2975,2024-01-01 00:45:00,2024-01-01 01:00:00,21.0
2976,2024-01-01 01:00:00,2024-01-01 01:15:00,20.0
...,...,...,...
5852,2024-01-31 00:00:00,2024-01-31 00:15:00,0.0
5853,2024-01-31 00:15:00,2024-01-31 00:30:00,0.0
5854,2024-01-31 00:30:00,2024-01-31 00:45:00,0.0
5855,2024-01-31 00:45:00,2024-01-31 01:00:00,0.0


totaal vebruik voor in jan berekenen

In [117]:
df_jan_totaal = df_jan['Verbruik in kW/h'].sum()
print(df_jan_totaal)

19037.0


Dan gaan nu met een query de profiel data en profielfractiesophalen.